# Introduction
In this notebook I am going to implement a Batch Gradient Descent with Early Stopping using Softmax Regression model <b>without Scikit-Learn</b> (or at least try to do it :D).

### Dataset
I will use Iris dataset from sklearn datasets

In [24]:
from sklearn import datasets

iris = datasets.load_iris()
X = iris["data"][:, 2:]
y = iris["target"]

As we can see, this dataset is sorted. If we would be using SGD there will be a need to shuffle it, but as we use BGD it may remain as it is.

### Softmax regression using BGD

In [25]:
import numpy as np

X_with_bias = np.c_[np.ones([len(X), 1]), X]
np.random.seed(2042)

In [26]:
theta1 = np.random.randn(2,1)
theta2 = np.random.randn(2,1)
theta3 = np.random.randn(2,1)

m = X.shape[0]
epochs = 100
eta = 0.1

for i, x in enumerate(X):
    s1 = x.T.dot(theta1)
    s2 = x.T.dot(theta2)
    s3 = x.T.dot(theta3)
    
    s = np.c_[s1, s2, s3].ravel()
    # print(s)
    
    p1 = np.exp(s1) / sum(np.exp(s))
    p2 = np.exp(s2) / sum(np.exp(s))
    p3 = np.exp(s3) / sum(np.exp(s))
    
    # print(p1, p2, p3)
    
    y_up = np.argmax(s)
    # print(y_up)
    
    yk1, yk2, yk3 = 0, 0, 0
    
    if y[i] == 0:
        yk1 = 1
    elif y[i] == 1:
        yk2 = 1
    elif y[i] == 2:
        yk3 = 1
    
    grad1 = 1/m * sum(p1 - yk1)*x
    grad2 = 1/m * sum(p2 - yk2)*x
    grad3 = 1/m * sum(p3 - yk3)*x
    
    theta1 = theta1 - eta*grad1
    theta2 = theta2 - eta*grad2
    theta3 = theta3 - eta*grad3
       

print(eta*grad1)
print(theta2)
print(theta3)
    

[3.51169627e-06 1.23942221e-06]
[[2.18765223 2.24309829]
 [0.59692038 0.65236644]]
[[ 0.11771323 -0.11642048]
 [ 0.99618503  0.76205132]]


### Proper Solution
I had a lot of trouble with this exercise so I decided to figure it out with the help of A. Geron github.

In [27]:
from sklearn import datasets

iris = datasets.load_iris()
X = iris["data"][:, 2:]
y = iris["target"]

We need to add the bias term for every instance (x0 = 1). Also let's set random seed to make output of this solution reproducible.

In [28]:
import numpy as np

X_with_bias = np.c_[np.ones([len(X), 1]), X]

np.random.seed(2042)

Let's split the dataset to train and test sets

In [48]:
test_ratio = 0.2
validation_ratio = 0.2
total_size = len(X_with_bias)

test_size = int(test_ratio * total_size)
validation_size = int(validation_ratio * total_size)
train_size = total_size - validation_size - test_size

# Shuffling the dataset
rnd_indices = np.random.permutation(total_size)

X_train = X_with_bias[rnd_indices[:train_size]]
y_train = y[rnd_indices[:train_size]]
X_valid = X_with_bias[rnd_indices[train_size:-test_size]]
y_valid = y[rnd_indices[train_size:-test_size]]
X_test = X_with_bias[rnd_indices[-test_size:]]
y_test = y[rnd_indices[-test_size:]]


Creating a one-hot vector of class probabilities.

In [30]:
def to_one_hot(y):
    n_classes = y.max() + 1
    m = len(y)
    
    Y_one_hot = np.zeros((m,n_classes))
    Y_one_hot[np.arange(m), y] = 1
    
    return Y_one_hot

print(y_train[:10])
print(to_one_hot(y_train[:10]))

[0 1 2 1 1 0 1 1 1 0]
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [1. 0. 0.]]


In [31]:
Y_train_one_hot = to_one_hot(y_train)
Y_valid_one_hot = to_one_hot(y_valid)
Y_test_one_hot = to_one_hot(y_test)

Now let's implement the sotfmax function

In [32]:
def softmax(logits):
    exps = np.exp(logits)
    exp_sums = np.sum(exps, axis=1, keepdims=True)
    
    return exps/exp_sums

In [33]:
n_inputs = X_train.shape[1] # == 3 (2 features plus the bias term)
n_outputs = len(np.unique(y_train))   # == 3 (3 iris classes)

Trainig the Sofmtax model

In [34]:
eta = 0.01
n_iterations = 5001
m = len(X_train)

epsilon = 1e-7 # to avoid nans while computing loss

Theta = np.random.randn(n_inputs, n_outputs)

for iteration in range(n_iterations):
    logits = X_train.dot(Theta)
    Y_proba = softmax(logits)
    
    if iteration % 500 == 0:
        loss = -np.mean(np.sum(Y_train_one_hot * np.log(Y_proba + epsilon), axis=1))
        print(iteration, loss)
    
    error = Y_proba - Y_train_one_hot
    gradients = 1/m * X_train.T.dot(error)
    
    Theta = Theta - eta * gradients

0 5.446205811872683
500 0.8350062641405651
1000 0.6878801447192402
1500 0.6012379137693313
2000 0.5444496861981872
2500 0.5038530181431525
3000 0.4729228972192248
3500 0.44824244188957774
4000 0.4278651093928793
4500 0.4106007142918712
5000 0.3956780375390374


In [35]:
Theta

array([[ 3.32094157, -0.6501102 , -2.99979416],
       [-1.1718465 ,  0.11706172,  0.10507543],
       [-0.70224261, -0.09527802,  1.4786383 ]])

Predictions on validation set

In [36]:
logits = X_valid.dot(Theta)
Y_proba = softmax(logits)
y_predict = np.argmax(Y_proba, axis=1)

acc_score = np.mean(y_predict == y_valid)
acc_score

0.9666666666666667

Let's add some l2 regularization and increase the learning rate.

In [37]:
eta = 0.1
n_iterations = 5001
m = len(X_train)
alpha = 0.1 # regularization hyperparameter

epsilon = 1e-7 # to avoid nans while computing loss

Theta = np.random.randn(n_inputs, n_outputs)

for iteration in range(n_iterations):
    logits = X_train.dot(Theta)
    Y_proba = softmax(logits)
    
    if iteration % 500 == 0:
        xentropy_loss = -np.mean(np.sum(Y_train_one_hot * np.log(Y_proba + epsilon), axis=1))
        l2_loss = 1/2 * np.sum(np.square(Theta[1:])) #dont regularize first element since it coreesponds to bias term
        loss = xentropy_loss + alpha*l2_loss
        print(iteration, loss)
    
    error = Y_proba - Y_train_one_hot
    gradients = 1/m * X_train.T.dot(error) + np.r_[np.zeros([1, n_outputs]), alpha*Theta[1:]]
    
    Theta = Theta - eta * gradients

0 6.629842469083912
500 0.5339667976629506
1000 0.503640075014894
1500 0.4946891059460321
2000 0.4912968418075477
2500 0.48989924700933296
3000 0.4892990598451198
3500 0.489035124439786
4000 0.4889173621830817
4500 0.4888643337449302
5000 0.4888403120738818


In [38]:
logits = X_valid.dot(Theta)
Y_proba = softmax(logits)
y_predict = np.argmax(Y_proba, axis=1)

accuracy_score = np.mean(y_predict == y_valid)
accuracy_score

1.0

Now lets add early stopping.

In [39]:
eta = 0.1 
n_iterations = 15001
m = len(X_train)
epsilon = 1e-7
alpha = 0.1  # regularization hyperparameter
best_loss = np.inf

Theta = np.random.randn(n_inputs, n_outputs)

for iteration in range(n_iterations):
    logits = X_train.dot(Theta)
    Y_proba = softmax(logits)
    error = Y_proba - Y_train_one_hot
    gradients = 1/m * X_train.T.dot(error) + np.r_[np.zeros([1, n_outputs]), alpha*Theta[1:]]
    
    Theta = Theta - eta * gradients
    
    logits = X_valid.dot(Theta)
    Y_proba = softmax(logits)
    xentropy_loss = -np.mean(np.sum(Y_valid_one_hot * np.log(Y_proba + epsilon), axis=1))
    l2_loss = 1/2 * np.sum(np.square(Theta[1:])) #dont regularize first element since it coreesponds to bias term
    loss = xentropy_loss + alpha*l2_loss
        
    if iteration % 500 == 0:
       print(iteration, loss)

    if loss < best_loss:
        best_loss = loss
    else:
        print(iteration-1, best_loss)
        print(iteration, loss, "early stopping!")
        break
    
    

0 4.7096017363419875
500 0.5739711987633519
1000 0.5435638529109127
1500 0.5355752782580262
2000 0.5331959249285544
2500 0.5325946767399383
2765 0.5325460966791898
2766 0.5325460971327977 early stopping!


In [40]:
logits = X_valid.dot(Theta)
Y_proba = softmax(logits)
y_predict = np.argmax(Y_proba, axis=1)

accuracy_score = np.mean(y_predict == y_valid)
accuracy_score

1.0

Let's measure the models accuracy on the test set.

In [50]:
logits = X_test.dot(Theta)
Y_proba = softmax(logits)
y_predict = np.argmax(Y_proba, axis=1)

accuracy_score = np.mean(y_predict == y_test)
accuracy_score

0.9333333333333333